# [BSSN](http://www2.yukawa.kyoto-u.ac.jp/~yuichiro.sekiguchi/3+1.pdf) Hamiltonian and momentum constraint equations, in ***curvilinear*** coordinates, using a covariant reference metric approach: C code generation

## Python module containing the final expressions: [BSSN/BSSN_Constraints.py](../edit/BSSN/BSSN_Constraints.py)

### ***Citations***: Generic curvilinear coordinate reference metric approach matches that of [Ruchlin, Etienne, and Baumgarte (2018)](https://arxiv.org/abs/1712.07658), which is an extension of the spherical coordinate reference metric approach of [Baumgarte, Montero, Cordero-Carrión, and Müller (2012)](https://arxiv.org/abs/1211.6632), which builds upon the covariant "Lagrangian" BSSN formalism of [Brown (2009)](https://arxiv.org/abs/0902.3652). *See also citations within each article.*

Eqs. 13 and 14 of [Baumgarte *et al.*](https://arxiv.org/pdf/1211.6632.pdf):
\begin{align}
H &= \frac{2}{3} K^2 - \bar{A}_{ij} \bar{A}^{ij} + e^{-4\phi} \left(\bar{R} - 8 \bar{D}^i \phi \bar{D}_i \phi - 8 \bar{D}^2 \phi\right) \\
\mathcal{M}^i &= e^{-4\phi} \left(\frac{1}{\bar{\gamma}} \hat{D}_j\left(\sqrt{\bar{\gamma}}\bar{A^{ij}}\right) + 6 \bar{A}^{ij}\partial_j \phi - \frac{2}{3} \bar{\gamma}^{ij}\partial_j K + \bar{A}^{jk} \Delta\Gamma^i_{jk}\right)
\end{align} 